# PV3 Visualisations of data gaps

<img src="http://reiner-lemoine-institut.de//wp-content/uploads/2015/09/rlilogo.png" width="100" style="float: right">

__copyright__ 	= "© Reiner Lemoine Institut" <br>
__license__ 	= "GNU Affero General Public License Version 3 (AGPL-3.0)" <br>
__url__ 		= "https://www.gnu.org/licenses/agpl-3.0.en.html" <br>
__author__ 		= "Ludwig Hülk" <br>

# Imports

In [1]:
import sys
import os
import getpass
import pandas as pd
import numpy as np
import json
from datetime import datetime
from sqlalchemy import *
# plot
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import plotly.graph_objs as go
import plotly.offline as pltly
import colorlover as cl
import seaborn as sns
# notebook
from IPython.display import Image
from IPython.core.display import HTML 

pltly.init_notebook_mode(connected=True)
%matplotlib inline

version = 'v1 (jupyter)'
project = 'pv3'

# Database Connection

In [ ]:
def postgres_session():
    """SQLAlchemy session object with valid connection to reeem database"""
    
    print('Please provide connection parameters to database:\n' +
              'Hit [Enter] to take defaults')
    host = 'localhost' # input('host (default 130.226.55.43): ')
    port = '5434' # input('port (default 5432): ')
    database = 'sonnja_db' # input("database name (default 'reeem'): ")
    user = 'sonnja' # input('user (default postgres): ')
    # password = input('password: ')
    password = getpass.getpass(prompt='password: ',
                                   stream=sys.stderr)
    con = create_engine(
            'postgresql://' + '%s:%s@%s:%s/%s' % (user,
                                                  password,
                                                  host,
                                                  port,
                                                  database)).connect()
    print('Password correct! Database connection established.')
    return con

# start session
con = postgres_session()

D:\Anaconda3\envs\d_py37_vis\lib\site-packages\ipykernel_launcher.py:12: UserWarning:

The `stream` parameter of `getpass.getpass` will have no effect when using ipykernel



Please provide connection parameters to database:
Hit [Enter] to take defaults


# Select related gaps
Requires script: htw_pv3_postgresql_07_data_analysis.sql

In [ ]:
# Select related gaps table
sql = text("""
    SELECT  *
    FROM    pv3.pv3_related_gaps
    """)
df_related_gaps = pd.read_sql_query(sql, con)
df_related_gaps

In [ ]:
# Get all days with gaps
df_related_gaps_days = df_related_gaps['date'].drop_duplicates().sort_values()
df_related_gaps_days

In [ ]:
# Select related gaps table
sql = text("""
    SELECT  *
    FROM    pv3.pv3_data_analysis_days_mview
    """)
df_related_gaps_days_mv = pd.read_sql_query(sql, con)
df_related_gaps_days_mv

# Select measurement data

Requires script: htw_pv3_postgresql_06_data_join.sql

In [ ]:
# Select metadata
sql = text("""SELECT obj_description('pv3.pv3_time_sun_weather_allwr_2015_mview'::regclass);""")
meta_str = pd.read_sql_query(sql, con).loc[0, 'obj_description']

# Select data
sql = text("""
    SELECT  *   -- column
    FROM    pv3.pv3_time_sun_weather_allwr_2015_mview  -- table
    """)
df = pd.read_sql_query(sql, con)
df = df.set_index('timestamp')
df.head()

# Metadata

The important information from the above select (**df**) is collected in a Dictionary (**info_dict**).

In [ ]:
# Facts dict
info_dict = {}
info_dict['Day'] = ['2015-04-25']
info_dict['Filename'] = ['{}_pv3_sonnja_plot_day_with_gap'.format(pd.to_datetime('today').strftime("%Y-%m-%d"))]
info_dict['Value'] = ['Leistung (P_AC)']
info_dict['Unit'] = ['W']
info_dict['Y-Axis'] = ['{} in {}'.format(*info_dict['Value'], *info_dict['Unit'])]
info_dict['X-Axis'] = ['Stunde im Jahr']
info_dict['Title'] = ['Tag mit Messlücke ({})'.format(*info_dict['Day'])]
info_dict['Metadata'] = [meta_str]

# Print facts
for x in info_dict:
    print(x,':',info_dict[x])

In [ ]:
# Filter day
day = datetime.fromisoformat('{}'.format(*info_dict['Day'])).date()
df_day = df[(df['date'] == day)]
df_day.head()

In [ ]:
df_plot = df_day[['wr1_p_ac']]

sns.set_palette("husl")
mfig = plt.figure(figsize=(12, 5))
ax = plt.subplot()
plt.title(*info_dict['Title'], fontsize=16)         # Title from info
plt.ylabel(*info_dict['Y-Axis'])                    # Label from info
df_plot.plot(ax=ax, lw=0.5, markeredgewidth=3)
ax.set_ylim(ymin=0)

## Interactive Plot with Metadata

In [ ]:
# Interactive plot
data = [
    go.Scatter(x=df_day.index.tolist(), 
               y=df_day['wr1_p_ac'].tolist(), 
               name='WR1', line=dict(), mode='lines'),
    go.Scatter(x=df_day.index.tolist(), 
               y=df_day['wr2_p_ac'].tolist(), 
               name='WR2', line=dict(), mode='lines'),
    go.Scatter(x=df_day.index.tolist(), 
               y=df_day['wr3_p_ac'].tolist(), 
               name='WR3', line=dict(), mode='lines'),
    go.Scatter(x=df_day.index.tolist(), 
               y=df_day['wr4_p_ac'].tolist(), 
               name='WR4', line=dict(), mode='lines'),
    go.Scatter(x=df_day.index.tolist(), 
               y=df_day['wr5_p_ac'].tolist(), 
               name='WR5', line=dict(), mode='lines'),
    go.Scatter(x=df_day.index.tolist(), 
               y=df_day['g_gen_cmp11'].tolist(), 
               name='CMP11', line=dict(), mode='lines'),
]
layout = go.Layout(
    title=''.join(info_dict['Title']),
    yaxis=dict(title=''.join(info_dict['Y-Axis'])),
    xaxis=dict(title=''.join(info_dict['X-Axis']))
)
ifig = go.Figure(data=data, layout=layout)
pltly.iplot(fig)

### One WR

## Save results to files

The results are saved to cooresponding files in a folder named **data**.<br>
The **info_dict** is saved to a text file (txt).<br>
The **data** is saved to a table (csv).<br>
The **plot** is saved as an image (png).<br>
The **interactive plot** is saved as a webpage (html). It will also open in a new tab of your browser.

In [ ]:
# Write facts to textfile
txtname = ('data/{}.txt').format(*info_dict['Filename'])
with open(txtname,'w') as tfile:
    for i in info_dict.keys():
        tfile.write(i + ": " + ', '.join([str(x) for x in info_dict[i]]) + "\n")
tfile.close()
print("Facts saved to file:", txtname)

# Save plot as file
plotname = 'data/{}.png' .format(*info_dict['Filename'])
mfig.savefig(plotname, bbox_inches='tight')
print("Plot saved to file:", plotname)

# Save interactive plot
htmlname = 'data/{}.html' .format(*info_dict['Filename'])
pltly.plot(ifig, filename=htmlname)
print("Interactive plot saved to file:", htmlname)